# 07_IDAO_Predict

* предсказываем значения на всех ранее изученных моделях
* сохраняем предсказания в форму **submission**

In [2]:
import sys
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.externals import joblib
import matplotlib.pylab as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as ltb

import seaborn as sns
sns.set(style="whitegrid")

In [14]:
features_x = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y']

features_y = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y']

features_z = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y']

features_Vx = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y'
            ]

features_Vy = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y'
            ]

features_Vz = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
            'sum_x_y', 'sum_x_z', 'sum_y_z',
            'dif_x_y', 'dif_x_z', 'dif_y_z',
            'mul_x_y', 'mul_x_z',
            'sum_Vx_Vy', 'sum_Vx_Vz', 'sum_Vy_Vz',
            'dif_Vx_Vy',
            'mul_Vx_Vy',
            'div_x_y'
            ]

In [15]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [5]:
smape_score = make_scorer(smape)

In [6]:
#Тестовые данные
df_test = pd.read_csv('data/Track 1/test_final.csv')

df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [7]:
df_test = df_test.drop(['time', 'date'], axis=1)
neworder = ['id', 'sat_id', 'x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim']
df_test = df_test.reindex(columns=neworder)
df_test['sat_id'] = df_test['sat_id'].astype('int64')

In [8]:
sat_id_test = list(set(df_test['sat_id']))

In [9]:
df_test.head(3)

,id,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,3927.0,1,-13579.030167,-14227.483997,6360.466650,4.307509,-0.730029,0.828370
1,NaN,1,-13296.178407,-14239.843338,6395.543856,4.342584,-0.680342,0.804909
2,NaN,1,-13013.326648,-14252.202678,6430.621062,4.377659,-0.630655,0.781448


In [10]:
def calc_smape(df):
    smape_x = smape(df['x'], df['x_sim'])
    smape_y = smape(df['y'], df['y_sim'])
    smape_z = smape(df['z'], df['z_sim'])
    
    smape_Vx = smape(df['Vx'], df['Vx_sim'])
    smape_Vy = smape(df['Vy'], df['Vy_sim'])
    smape_Vz = smape(df['Vz'], df['Vz_sim'])    
    print('sMAPE x:', smape_x)
    print('sMAPE y:', smape_y)
    print('sMAPE z:', smape_z)
    print()
    
    print('sMAPE Vx:', smape_Vx)
    print('sMAPE Vy:', smape_Vy)
    print('sMAPE Vz:', smape_Vz)
    print()
    
    final_smape = (smape_x + smape_y + smape_z + smape_Vx + smape_Vy + smape_Vz) / 6
    print('smape Final', final_smape)

In [16]:
def features_engeneering(df_one_sat):
    df_one_sat['sum_x_y'] = df_one_sat['x_sim'] + df_one_sat['y_sim']
    df_one_sat['sum_x_z'] = df_one_sat['x_sim'] + df_one_sat['z_sim']
    df_one_sat['sum_y_z'] = df_one_sat['y_sim'] + df_one_sat['z_sim']

    df_one_sat['dif_x_y'] = df_one_sat['x_sim'] - df_one_sat['y_sim']
    df_one_sat['dif_x_z'] = df_one_sat['x_sim'] - df_one_sat['z_sim']
    df_one_sat['dif_y_z'] = df_one_sat['y_sim'] - df_one_sat['z_sim']

    df_one_sat['mul_x_y'] = df_one_sat['x_sim'] * df_one_sat['y_sim']
    df_one_sat['mul_x_z'] = df_one_sat['x_sim'] * df_one_sat['z_sim']
    df_one_sat['mul_y_z'] = df_one_sat['y_sim'] * df_one_sat['z_sim']

    df_one_sat['div_x_y'] = df_one_sat['x_sim'] / df_one_sat['y_sim']
    df_one_sat['div_x_z'] = df_one_sat['x_sim'] / df_one_sat['z_sim']
    df_one_sat['div_y_z'] = df_one_sat['y_sim'] / df_one_sat['z_sim']

    df_one_sat['sum_Vx_Vy'] = df_one_sat['Vx_sim'] + df_one_sat['Vy_sim']
    df_one_sat['sum_Vx_Vz'] = df_one_sat['Vx_sim'] + df_one_sat['Vz_sim']
    df_one_sat['sum_Vy_Vz'] = df_one_sat['Vy_sim'] + df_one_sat['Vz_sim']

    df_one_sat['dif_Vx_Vy'] = df_one_sat['Vx_sim'] - df_one_sat['Vy_sim']
    df_one_sat['dif_Vx_Vz'] = df_one_sat['Vx_sim'] - df_one_sat['Vz_sim']
    df_one_sat['dif_Vy_Vz'] = df_one_sat['Vy_sim'] - df_one_sat['Vz_sim']

    df_one_sat['mul_Vx_Vy'] = df_one_sat['Vx_sim'] * df_one_sat['Vy_sim']
    df_one_sat['mul_Vx_Vz'] = df_one_sat['Vx_sim'] * df_one_sat['Vz_sim']
    df_one_sat['mul_Vy_Vz'] = df_one_sat['Vy_sim'] * df_one_sat['Vz_sim']
    return df_one_sat

In [17]:
def get_predictions(df_test, sat_id_test, features_x, features_y):
    
    df_answ = df_test.copy()
    df_result = pd.DataFrame(columns=['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz',
                                      'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'])
    
    for sat_id in sat_id_test:
        print('Sat:', sat_id)
        df_test_one_sat = df_test.query('sat_id == @sat_id')
        df_test_one_sat = features_engeneering(df_test_one_sat)
        
        features_test_x = df_test_one_sat[features_x]
        features_test_y = df_test_one_sat[features_y]
        features_test_z = df_test_one_sat[features_z]
        
        features_test_Vx = df_test_one_sat[features_Vx]
        features_test_Vy = df_test_one_sat[features_Vy]
        features_test_Vz = df_test_one_sat[features_Vz]
        
        model_path_x = 'models/minute_x/lgb_x_' + str(sat_id) + '.pkl'
        model_path_y = 'models/minute_y/lgb_y_' + str(sat_id) + '.pkl'
        model_path_z = 'models/minute_z/lgb_z_' + str(sat_id) + '.pkl'
        
        model_path_Vx = 'models/minute_Vx/lgb_Vx_' + str(sat_id) + '.pkl'
        model_path_Vy = 'models/minute_Vy/lgb_Vy_' + str(sat_id) + '.pkl'
        model_path_Vz = 'models/minute_Vz/lgb_Vz_' + str(sat_id) + '.pkl'        
        
        # load model
        model_LTB_x = joblib.load(model_path_x)
        model_LTB_y = joblib.load(model_path_y)
        model_LTB_z = joblib.load(model_path_z)
        
        model_LTB_Vx = joblib.load(model_path_Vx)
        model_LTB_Vy = joblib.load(model_path_Vy)
        model_LTB_Vz = joblib.load(model_path_Vz)  
        
        df_test_one_sat['x'] = model_LTB_x.predict(features_test_x)
        df_test_one_sat['y'] = model_LTB_y.predict(features_test_y)
        df_test_one_sat['z'] = model_LTB_z.predict(features_test_z)
        
        df_test_one_sat['Vx'] = model_LTB_Vx.predict(features_test_Vx)
        df_test_one_sat['Vy'] = model_LTB_Vy.predict(features_test_Vy)
        df_test_one_sat['Vz'] = model_LTB_Vz.predict(features_test_Vz)
        
        
        
        df_result = pd.concat([df_result, df_test_one_sat])  
    return df_result    

In [18]:
df_final = get_predictions(df_test, sat_id_test, features_x, features_y)

Sat: 1
Sat: 514
Sat: 2
Sat: 3
Sat: 4
Sat: 6
Sat: 515
Sat: 516
Sat: 9
Sat: 517
Sat: 519
Sat: 522
Sat: 523
Sat: 524
Sat: 16
Sat: 528
Sat: 20
Sat: 22
Sat: 24
Sat: 25
Sat: 26
Sat: 27
Sat: 28
Sat: 29
Sat: 536
Sat: 537
Sat: 32
Sat: 539
Sat: 34
Sat: 35
Sat: 36
Sat: 37
Sat: 38
Sat: 39
Sat: 40
Sat: 41
Sat: 42
Sat: 547
Sat: 44
Sat: 45
Sat: 549
Sat: 550
Sat: 558
Sat: 49
Sat: 51
Sat: 52
Sat: 53
Sat: 54
Sat: 563
Sat: 565
Sat: 57
Sat: 566
Sat: 59
Sat: 567
Sat: 569
Sat: 572
Sat: 63
Sat: 64
Sat: 575
Sat: 578
Sat: 579
Sat: 68
Sat: 580
Sat: 70
Sat: 583
Sat: 584
Sat: 75
Sat: 587
Sat: 588
Sat: 593
Sat: 82
Sat: 595
Sat: 84
Sat: 596
Sat: 86
Sat: 597
Sat: 89
Sat: 90
Sat: 91
Sat: 92
Sat: 93
Sat: 96
Sat: 98
Sat: 100
Sat: 101
Sat: 102
Sat: 105
Sat: 106
Sat: 109
Sat: 110
Sat: 112
Sat: 113
Sat: 114
Sat: 117
Sat: 118
Sat: 120
Sat: 121
Sat: 123
Sat: 124
Sat: 125
Sat: 126
Sat: 127
Sat: 128
Sat: 130
Sat: 132
Sat: 133
Sat: 135
Sat: 138
Sat: 142
Sat: 143
Sat: 144
Sat: 145
Sat: 149
Sat: 151
Sat: 152
Sat: 153
Sat: 155
Sa

In [21]:
df_result = df_final[df_final['id'].notna()]

In [23]:
calc_smape(df_result)

sMAPE x: 0.3000268036992722
sMAPE y: 0.30341361776150716
sMAPE z: 0.3008523640891167

sMAPE Vx: 0.31101119810291994
sMAPE Vy: 0.3176248403860361
sMAPE Vz: 0.3137035512354

smape Final 0.3077720625457087


In [24]:
columns = ['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']
df_result = df_result[columns]
df_result['id'] = df_result['id'].astype('int64')

df_result = df_result.reset_index()
df_result = df_result.drop(['index'], axis=1)

In [27]:
df_result

,id,x,y,z,Vx,Vy,Vz
0,3927,-25357.687470,-10951.052784,6526.241679,2.747672,-1.330908,1.054990
1,3928,-22267.036307,-12357.981021,7665.999908,3.132560,-0.986903,0.918266
2,3929,-14295.578770,-13482.724203,8987.424567,4.166048,-0.209204,0.378675
3,3930,-4944.012646,-12866.559263,8913.614131,5.203153,1.280659,-0.603036
4,3931,905.274254,-10300.578196,7416.022376,5.398223,3.067529,-1.819854
...,...,...,...,...,...,...,...
284066,1049840,-35979.805726,-3056.063977,-2500.551681,-0.677578,-0.745324,1.582006
284067,1049841,-39388.355999,-4834.096377,-465.141417,-0.226268,-0.780536,1.727729
284068,1049842,-39855.526546,-5973.789094,3065.101070,0.311219,-0.633621,1.646334
284069,1049843,-39037.222975,-7943.125923,6403.807649,0.916227,-0.576119,1.631377


In [28]:
df_result.to_csv('data/Track 1/submission_full.csv', index=False)